In [1]:
#!pip install --force-reinstall -v "numpy==1.19.5"

Using pip 22.3.1 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)
  Link requires a different Python (3.8.10 not in: '>=3.9'): https://files.pythonhosted.org/packages/26/de/437a60a69f7fd0c79264530a97787b2ac7394616e3661236201518f8a47d/numpy-1.25.0rc1.tar.gz (from https://pypi.org/simple/numpy/) (requires-python:>=3.9)
  Link requires a different Python (3.8.10 not in: '>=3.9'): https://files.pythonhosted.org/packages/d0/b2/fe774844d1857804cc884bba67bec38f649c99d0dc1ee7cbbf1da601357c/numpy-1.25.0.tar.gz (from https://pypi.org/simple/numpy/) (requires-python:>=3.9)
  Link requires a different Python (3.8.10 not in: '>=3.9'): https://files.pythonhosted.org/packages/cf/7a/f68d1d658a0e68084097beb212fa9356fee7eabff8b57231cc4acb555b12/numpy-1.25.1.tar.gz (from https://pypi.org/simple/numpy/) (requires-python:>=3.9)
  Link requires a different Python (3.8.10 not in: '>=3.9'): https://files.pythonhosted.org/packages/a0/41/8f53eff8e969dd8576ddfb45e7ed315407d27c7518ae49418be8ed532b07/n

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

ImportError: Matplotlib requires numpy>=1.20; you have 1.19.5

In [ ]:
import os
os.chdir('../../')

In [ ]:
import fastai.text.models.transformer

In [ ]:
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *

# Generate Music with Pretrained Model

### Load Pretrained

In [ ]:
# Location of your midi files
midi_path =  Path('data/midi/examples')

# Location of saved datset
data_path = Path('data/numpy')

In [ ]:
# Data
data = MusicDataBunch.empty(data_path)
vocab = data.vocab

# For Saved Data:
# data = load_data(data_path, 'musicitem_data_save.pkl')

In [ ]:
# Pretrained Model
# Download pretrained model if you haven't already
pretrained_url = 'https://ashaw-midi-web-server.s3-us-west-2.amazonaws.com/pretrained/MusicTransformerKeyC.pth'
# pretrained_url = 'https://ashaw-midi-web-server.s3-us-west-2.amazonaws.com/pretrained/MusicTransformer.pth'

pretrained_path = data_path/'pretrained'/Path(pretrained_url).name
download_url(pretrained_url, dest=pretrained_path)

In [ ]:
# Learner
learn = music_model_learner(data, pretrained_path=pretrained_path)

## Prediction

#### Choose existing midi file as a starting point

In [ ]:
midi_files = get_files(midi_path, recurse=True, extensions='.mid'); midi_files[:4]

In [ ]:
idx = 1
f = midi_files[idx]; f

#### NextWord/Autocomplete

Trim the song to only a few notes. Model will use these notes a seed and continue the idea

In [ ]:
cutoff_beat = 10

item = MusicItem.from_file(f, data.vocab)
seed_item = item.trim_to_beat(cutoff_beat)

In [ ]:
seed_item.show()

In [ ]:
seed_item.play()

#### Use seed to predict next sequence

In [ ]:
pred, full = learn.predict(seed_item, n_words=400, temperatures=(1.1,0.4), min_bars=12, top_k=24, top_p=0.7)

In [ ]:
pred.show()

In [ ]:
pred.play()

In [ ]:
full_song = seed_item.append(pred); full_song.show()

In [ ]:
full_song.play()

#### Add More Randomness to prediction

In [ ]:
note_temp = 1.4 # Determines amount of variation in note pitches
dur_temp = 0.8 # Amount of randomness in rhythm
top_k = 30
pred, full = learn.predict(seed_item, n_words=400, temperatures=(note_temp, dur_temp), min_bars=12, top_k=top_k, top_p=0.7)

In [ ]:
pred.show()

### Pop Music Theory

According to hooktheory, the most popular chord progression is I-V-vi-IV  
https://www.hooktheory.com/theorytab/common-chord-progressions

In [ ]:
# Let's create a partial progression I-V-vi
p = music21.stream.Part()
p.append(music21.chord.Chord('C4 E4 G4', type='half')) # I
p.append(music21.chord.Chord('G3 B3 D4', type='half')) # V
p.append(music21.chord.Chord('A3 C4 E4', type='half')) # vi
s = music21.stream.Score([p])
chord_item = MusicItem.from_stream(s, vocab)
chord_item.show()

In [ ]:
temperaturs = (0.5,0.5) # Let's lower the note randomness for this test
pred, full = learn.predict(chord_item, n_words=10, temperatures=(0.5,0.5))
full.show()

In [ ]:
# Predicted chords - IV
pred.show()

In [ ]:
# Full sequence
chord_item.append(pred).show()

Looks like it predicted the most popular progression!

#### Predict without a starting sequence

In [ ]:
empty_item = MusicItem.empty(vocab)

In [ ]:
pred, full = learn.predict(empty_item, n_words=200)

In [ ]:
pred.show()